In [44]:
import pandas as pd
import requests
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

launch_url = "https://api.spacexdata.com/v4/launches"
pad_url = "https://api.spacexdata.com/v4/launchpads"

launches = pd.json_normalize(requests.get(launch_url).json())
pads = pd.json_normalize(requests.get(pad_url).json())

pads = pads[['id', 'name']].rename(columns={'name': 'launchpad_name'})
launches = launches[['name', 'date_utc', 'success', 'launchpad']]

df = pd.merge(launches, pads, left_on='launchpad', right_on='id')
df['date_utc'] = pd.to_datetime(df['date_utc'])
df['year_month'] = df['date_utc'].dt.to_period('M').astype(str)

app = Dash(__name__)
app.title = "SpaceX Launch Dashboard"

app.layout = html.Div([
    html.H2("SpaceX Launch Dashboard", style={'textAlign': 'center'}),
    
    html.Div([
        html.Label("Select Launchpad:"),
        dcc.Dropdown(
            options=[{"label": name, "value": name} for name in df['launchpad_name'].unique()],
            value=None,
            id='pad-dropdown',
            placeholder="All Launchpads",
            clearable=True
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed=df['date_utc'].min().date(),
            max_date_allowed=df['date_utc'].max().date(),
            start_date=df['date_utc'].min().date(),
            end_date=df['date_utc'].max().date()
        )
    ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),

    html.Br(), html.Hr(),

    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='pie-chart'),
    dcc.Graph(id='line-chart')
])

@app.callback(
    [Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('line-chart', 'figure')],
    [Input('pad-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_charts(selected_pad, start_date, end_date):
    filtered = df.copy()
    if selected_pad:
        filtered = filtered[filtered['launchpad_name'] == selected_pad]
    if start_date and end_date:
        filtered = filtered[
            (filtered['date_utc'] >= pd.to_datetime(start_date)) &
            (filtered['date_utc'] <= pd.to_datetime(end_date))
        ]

    bar_fig = px.bar(
        filtered.groupby('launchpad_name').size().reset_index(name='Launch Count'),
        x='launchpad_name', y='Launch Count', title="Launches per Launchpad"
    )

    pie_fig = px.pie(
        filtered, names='success', title="Success vs Failure"
    )

    line_fig = px.line(
        filtered.groupby('year_month').size().reset_index(name='Launches'),
        x='year_month', y='Launches', title="Launches Over Time"
    )

    return bar_fig, pie_fig, line_fig

if __name__ == "__main__":
    app.run(debug=True)


C:\Users\kash7\AppData\Local\Temp\ipykernel_5008\1830144414.py:19: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File ~\AppData\Roaming\Python\Python312\site-packages\pandas\core\arrays\datetimelike.py:536, in DatetimeLikeArrayMixin._validate_comparison_value(
    self=<DatetimeArray>
['2006-03-24 22:30:00+00:00', '2...0+00:00']
Length: 205, dtype: datetime64[ns, UTC],
    other=Timestamp('2006-03-24 00:00:00')
)
    535 try:
--> 536     self._check_compatible_with(other)
        other = Timestamp('2006-03-24 00:00:00')
        self = <DatetimeArray>
['2006-03-24 22:30:00+00:00', '2007-03-21 01:10:00+00:00',
 '2008-08-03 03:34:00+00:00', '2008-09-28 23:15:00+00:00',
 '2009-07-13 03:35:00+00:00', '2010-06-04 18:45:00+00:00',
 '2010-12-08 15:43:00+00:00', '2012-05-22 07:44:00+00:00',
 '2012-10-08 00:35:00+00:00', '2013-03-01 19:10:00+00:00',
 ...
 '2022-11-18 22:00:00+00:00', '2022-11-01 00:00:00+00:00',
 '2022-11-01 00:00:00+00:00', '2022-12-05 00: